# pip安装transformers

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.0 MB/s 
     |████████████████████████████████| 636 kB 46.8 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
     |████████████████████████████████| 3.3 MB 28.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 2.2.1 视频学习

## stage1，Tokenizer过程的理解

In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # get from https://huggingface.co/models
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


## stage2，模型化的理解

In [7]:
from transformers import AutoTokenizer, AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # get from https://huggingface.co/models
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")


model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

print(outputs)


Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 16, 768])
BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[-0.3088,  0.7332, -0.1861,  ..., -0.1305, -0.9360, -0.0433],
         [-0.3340,  0.9830, -0.0946,  ..., -0.3825, -0.6176,  0.2008],
         [-0.1687,  0.8781, -0.1117,  ..., -0.2380, -0.7790,  0.0935],
         ...,
         [-0.3047,  0.7850, -0.2043,  ..., -0.1101, -0.7665, -0.0563],
         [-0.4014,  0.8999, -0.2245,  ..., -0.1877, -0.6910,  0.0329],
         [-0.2933,  0.7947, -0.2140,  ..., -0.1002, -0.7508, -0.0623]]],
       grad_fn=<NativeLayerNormBackward>), hidden_states=

每个AutoModelForXxx的类加载一个适应于指定任务的模型

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # get from https://huggingface.co/models
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")


model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs)

print(outputs.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.2141, -3.4158]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
tensor([[-1.5607,  1.6123],
        [ 4.2141, -3.4158]], grad_fn=<AddmmBackward>)


## stage3

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # get from https://huggingface.co/models
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")


model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=1) # 为什么-1，实际上1也可以，
print(predictions)

print(model.config.id2label)


print(model.config.id2label[0], float(predictions[0][0])) # 第0句话的0id的概率

tensor([[4.0195e-02, 9.5980e-01],
        [9.9951e-01, 4.8549e-04]], grad_fn=<SoftmaxBackward>)
{0: 'NEGATIVE', 1: 'POSITIVE'}
NEGATIVE 0.04019518569111824


# 2.2.5 try it out

Choose two (or more) texts of your own and run them through the sentiment-analysis pipeline. Then replicate the steps you saw here yourself and check that you obtain the same results!

pipeline

In [23]:
from transformers import pipeline

classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
classifier([
    "Tony Tom Jerry Jim", # {'label': 'NEGATIVE', 'score': 0.7907747030258179}，置信度不足0.9的时候很可能就很不可靠了
    "Wow, I'm so happy about this tour guide!",
    "So tired! I can't wait to take a break"
])

[{'label': 'NEGATIVE', 'score': 0.7907747030258179},
 {'label': 'POSITIVE', 'score': 0.9998595118522644},
 {'label': 'NEGATIVE', 'score': 0.9996296763420105}]

stage-by-stage

In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch  

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

raw_inputs = [
    "Tony Tom Jerry Jim",
    "Wow, I'm so happy about this tour guide!",
    "So tired! I can't wait to take a break"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=1)

for i in range(len(raw_inputs)):
    print("Text:" + raw_inputs[i], end='\t')
    for key, value in model.config.id2label.items():
        print(value + '置信度：' + str(float(predictions[i][key])), end='\t')
    print("")



Text:Tony Tom Jerry Jim	NEGATIVE置信度：0.7907747626304626	POSITIVE置信度：0.20922529697418213	
Text:Wow, I'm so happy about this tour guide!	NEGATIVE置信度：0.0001404868089593947	POSITIVE置信度：0.9998594522476196	
Text:So tired! I can't wait to take a break	NEGATIVE置信度：0.9996296167373657	POSITIVE置信度：0.00037032406544312835	
